In [66]:
#!/usr/bin/env python3
"""
OpenJordi Column Mapper

This script maps columns from downloaded grant data files to the CrossRef grant ontology.
It follows a similar interface to fetch_data.py.

Usage:
    python map_columns.py --source SOURCE_ID     # Map columns for a specific source
    python map_columns.py --force                # Force remapping even if mapping exists
    python map_columns.py --all                  # Map all sources
"""

import os
import sys
import json
import logging
import pandas as pd
import hashlib
from pathlib import Path
from datetime import datetime
import openai
from dotenv import load_dotenv
from together import Together


# Add project root to path to ensure imports work correctly
# Add the 'config' directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'ontology')))
sys.path.append(os.path.abspath(os.path.join('..', 'config')))
sys.path.append(os.path.abspath(os.path.join('..', 'data')))

sys.path.append('..')

# Import configuration
LLM_MODEL = "gpt-4"
LOG_LEVEL = '1'
LOG_FILE = 'a'
LLM_API_KEY = None
# Import schema definition

from grant_ontology import crossref_metadata

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("openjordi.map_columns")

# Ensure output directories exist
ONTOLOGY_DIR = os.path.join('..', "ontology")
MAPPINGS_DIR = os.path.join(ONTOLOGY_DIR, "mappings")
os.makedirs(MAPPINGS_DIR, exist_ok=True)


"""Initialize the column mapper."""
# Load environment variables
load_dotenv()

LLM_PROVIDER = 'openai'
# Setup API key for LLM
llm_provider = LLM_PROVIDER
api_key = LLM_API_KEY or os.environ.get("OPENAI_API_KEY") or os.environ.get("SAMBANOVA_API_KEY")
model = LLM_MODEL

if not api_key:
    logger.error("No API key found for LLM. Set OPENAI_API_KEY or SAMBANOVA_API_KEY in environment.")
    raise ValueError("Missing API key for language model")

# Initialize OpenAI client based on provider
if llm_provider.lower() == "togetherai":
    client = Together(
        api_key=os.environ.get("TOGETHER_AI_API_KEY"),
    )
else:  # Default to OpenAI
    client = openai.OpenAI(api_key=api_key)

In [144]:
file_path = '../data/raw/Gates Foundation/Committed Grants/latest/bmgf-grants.csv'#'../data/raw/Arcadia/latest/WebData-08-2024.csv'
# Try multiple approaches to handle problematic CSVs
try:
    df = pd.read_csv(file_path, skip_blank_lines=True)
except Exception as e:
    logger.warning(f"Standard CSV reading failed: {str(e)}")
    # Try with error handling options
    try:
        if pd.__version__ >= '1.3.0':
            df = pd.read_csv(file_path, sep=';',on_bad_lines='skip',skip_blank_lines=True, header=1)
        else:
            df = pd.read_csv(file_path, error_bad_lines=False,skip_blank_lines=True,skiprows=1)
    except Exception:
        # Last resort: try with different encoding
        df = pd.read_csv(file_path, encoding='latin1', 
                        on_bad_lines='skip' if pd.__version__ >= '1.3.0' else None)

In [150]:
df = pd.read_csv(file_path, sep=',',on_bad_lines='skip',skip_blank_lines=True, header=1)
df = df.dropna(how='all')


In [157]:
source_name = 'Gates Foundation'

In [158]:
"""
Use an LLM to map source columns to CrossRef schema.

Args:
    columns (list): List of column names to map
    source_name (str): The name of the source
    
Returns:
    dict: Mapping from source columns to schema columns
"""

columns = df.columns
column_examples = {}

for col in columns:
    if col in df.columns:
        if not df[col].iloc[0] is None and not pd.isna(df[col].iloc[0]):
            column_examples[col] = str(df[col].iloc[0])
        else:
            break

# Format column information with examples
column_info = []
for col in columns:
    if col in column_examples:
        column_info.append(f'* "{col}": {column_examples.get(col)}')
    else:
        column_info.append(f'* "{col}"')

column_text = "\n".join(column_info)


# Prepare schema information with descriptions
schema_info = []
for field, details in crossref_metadata.items():
    schema_info.append(f"- {field}: {details['Description']} ({details['Limits']})")

schema_text = "\n".join(schema_info)

# Create prompt for LLM
prompt = f"""
You are an expert in data schema mapping for academic grant data.

I need to map columns from a dataset about research grants funded by '{source_name}' to the CrossRef grant metadata schema.

SOURCE COLUMNS (with examples from first row if available):
{column_text}

TARGET SCHEMA (CrossRef grant metadata):
{schema_text}

For each source column, map it to the most appropriate CrossRef schema field, or 'null' if there is no appropriate match.
Consider semantic meaning, not just exact name matches. Be thorough and consider all possible mappings.

Return your response as a valid json with the following format:
{{
    "source_column_name": "crossref_field_name",
    ...
}}

Only include the Python dictionary in your response, with no additional text.
"""


In [153]:
def _clean_llm_response(response_text):
    """
    Clean the LLM response to ensure it's a valid Python dictionary.
    
    Args:
        response_text (str): The raw response from the LLM
        
    Returns:
        str: Cleaned response text that can be safely evaluated
    """
    # Remove markdown code blocks if present
    if response_text.startswith("```json"):
        response_text = response_text[9:]
    elif response_text.startswith("```"):
        response_text = response_text[3:]
    
    if response_text.endswith("```"):
        response_text = response_text[:-3]
    
    response_text = response_text.strip()
    
    # Ensure it starts and ends with curly braces
    if not response_text.startswith("{"):
        response_text = "{" + response_text
    if not response_text.endswith("}"):
        response_text = response_text + "}"
    
    return response_text

In [159]:
response = client.chat.completions.create(
model=model,
messages=[
    {"role": "system", "content": "You are a helpful academic data assistant that maps columns between schemas."},
    {"role": "user", "content": prompt}
],
temperature=0.1,
max_tokens=1500,
)

# Get the response content
response_text = response.choices[0].message.content.strip()

In [160]:
print(_clean_llm_response(response_text))

{
    "GRANT ID": "award-number",
    "GRANTEE": "funder-name",
    "PURPOSE": "description",
    "DIVISION": "funding-scheme",
    "DATE COMMITTED": "award-start-date",
    "DURATION (MONTHS)": null,
    "AMOUNT COMMITTED": "award-amount",
    "GRANTEE WEBSITE": "resource",
    "GRANTEE CITY": null,
    "GRANTEE STATE": null,
    "GRANTEE COUNTRY": "@country",
    "REGION SERVED": null,
    "TOPIC": "project-title"
}


In [156]:
print(prompt)


You are an expert in data schema mapping for academic grant data.

I need to map columns from a dataset about research grants from 'Sao Paolo' to the CrossRef grant metadata schema.

SOURCE COLUMNS (with examples from first row if available):
* "GRANT ID": INV-002690
* "GRANTEE": World Health Organization
* "PURPOSE": to reduce cholera's disease burden in both epidemic and endemic settings through use of evidence-based practices, increased stakeholder collaboration and an execution plan linked to the Ending Cholera - Global Roadmap to 2030
* "DIVISION": Global Health
* "DATE COMMITTED": 2021-02
* "DURATION (MONTHS)": 16.0
* "AMOUNT COMMITTED": 1078614.0
* "GRANTEE WEBSITE": http://www.who.int
* "GRANTEE CITY": Geneva
* "GRANTEE STATE"
* "GRANTEE COUNTRY"
* "REGION SERVED"
* "TOPIC"

TARGET SCHEMA (CrossRef grant metadata):
- project: Container for project information. Multiple projects may be assigned to a single Grant ID. (required; multiple allowed)
- project-title: Title of a proje

In [ ]:
import os
import pandas as pd
import json
import sys
import openai
from dotenv import load_dotenv
from together import Together

# Load environment variables from .env file
load_dotenv()

# Add the 'config' directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'ontology')))
sys.path.append(os.path.abspath(os.path.join('..', 'data')))
from grant_ontology import crossref_metadata  # Import schema definition

# Define the data folders
RAW_FOLDER = os.path.abspath(os.path.join("..", "data", "raw"))
CLEANED_FOLDER = "../ontology/mappings/"
os.makedirs(CLEANED_FOLDER, exist_ok=True)

import re
import re
import json

def extract_json_from_text(text):
    try:
        # Use regex to find JSON content
        match = re.search(r'```json\n(.*?)\n```', text, re.DOTALL)
        if match:
            json_str = match.group(1)  # Extract JSON string
            return json.loads(json_str)  # Parse JSON
        else:
            return None  # No JSON found
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None


def generate_prompt(example_prompt, crossref_metadata):
  my_prompt = f'''Persona:
You are a meticulous data analyst specializing in research funding metadata. Your expertise lies in mapping datasets to standardized schemas while ensuring high accuracy and logical consistency. You strictly adhere to the given schema constraints, avoiding any modifications or extensions beyond the defined attributes. You carefully evaluate both column names and example values to determine the most appropriate mapping.

Task:
Given the following dataset columns, map them to the closest attribute in the CrossRef metadata schema. Only select attributes that have the most similar meaning based on both column names and example values. The CrossRef schema cannot be extended.

Return the mapping as a JSON object, where keys represent dataset column names and values represent the corresponding CrossRef schema attributes.

Dataset:
{str(example_prompt)}

CrossRef Schema:
{str(crossref_metadata)}

Output Format:

```json
{{
  "DatasetColumn1": "CrossRefAttribute1",
  "DatasetColumn2": "CrossRefAttribute2",
  ...
}}```

Ensure that:

Only the most relevant attributes are mapped.
Attributes that do not closely match are excluded.
The response strictly follows JSON formatting.
'''

  return my_prompt



def get_column_mapping(db_schema, schema):
    """Use LLM to generate a mapping between raw data columns and Crossref schema."""

    crossref_metadata = "{"+ "\n".join(f'"{key}": "{value["Description"]}"' for key, value in schema.items()) + "}"

    prompt = generate_prompt(db_schema, crossref_metadata)

    client = Together(
        api_key=os.environ.get("TOGETHER_AI_API_KEY"),
    )

    response = client.chat.completions.create(
                    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
                    messages=[{"role":"system","content":prompt}],
                    temperature=0.1,
                    top_p=0.1,
                    top_k=50,
                    repetition_penalty=1,
                    stop=["<|eot_id|>","<|eom_id|>"],
                    stream=False
                )

    mapping = response.choices[0].message.content
    return extract_json_from_text(mapping)  # Convert response string to dictionary

def normalize_columns(df, mapping):
    """Rename columns based on LLM-provided mapping."""
    return df.rename(columns=mapping)

def clean_data():
    """Clean and structure downloaded grant data based on Crossref schema."""
    
if __name__ == "__main__":
    clean_data()


ModuleNotFoundError: No module named 'config'

In [ ]:
load_dotenv()
        
        # Setup API key for LLM
        llm_provider = LLM_PROVIDER
        api_key = LLM_API_KEY or os.environ.get("OPENAI_API_KEY") or os.environ.get("SAMBANOVA_API_KEY")
        model = LLM_MODEL
        
        if not api_key:
            logger.error("No API key found for LLM. Set OPENAI_API_KEY or SAMBANOVA_API_KEY in environment.")
            raise ValueError("Missing API key for language model")
        
        # Initialize OpenAI client based on provider
        if llm_provider.lower() == "sambanova":
            client = openai.OpenAI(
                api_key=api_key,
                base_url="https://api.sambanova.ai/v1",
            )
        else:  # Default to OpenAI
            client = openai.OpenAI(api_key=api_key)

Processing: /home/siris/Repositories/openjordi/data/raw/HRCS/HRCS.xlsx
✅ Cleaned data saved: ../ontology/mappings/HRCS.json
Processing: /home/siris/Repositories/openjordi/data/raw/Swiss NSF/swiss_nsf.csv
❌ Error processing swiss_nsf.csv: read_csv() got an unexpected keyword argument 'error_bad_lines'
Processing: /home/siris/Repositories/openjordi/data/raw/Arcadia/arcadia.csv
❌ Error processing arcadia.csv: read_csv() got an unexpected keyword argument 'error_bad_lines'


In [127]:
cleaned_file_path = os.path.join(CLEANED_FOLDER, f"{source_folder}.json")
with open(cleaned_file_path, "w", encoding="utf-8") as f:
    json.dump(mapping, f, indent=4)
print(f"✅ Cleaned data saved: {cleaned_file_path}")
            

✅ Cleaned data saved: ../ontology/mappings/HRCS.json


In [102]:
# # to provide this example as part of the prompt with column name and an example value
# example_prompt = generate_prompt(df)



# print(prompt)

# query the model
client = Together(
        api_key=os.environ.get("TOGETHER_AI_API_KEY"),
    )

response = client.chat.completions.create(
                model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
                messages=[{"role":"system","content":my_prompt}],
                temperature=0.1,
                top_p=0.1,
                top_k=50,
                repetition_penalty=1,
                stop=["<|eot_id|>","<|eom_id|>"],
                stream=False
            )

mapping = response.choices[0].message.content
mapping

'<think>\nAlright, I\'m trying to map the given dataset columns to the CrossRef metadata schema. Let me go through each column one by one and see where they fit best.\n\nStarting with "HRCS2022_ID". This looks like a unique identifier for the project. In the CrossRef schema, there\'s "award-number" which is the funder-supplied award ID. That seems like a perfect match.\n\nNext is "FundingOrganisation". This is the name of the organization providing the funds. The closest attribute in CrossRef is "funder-name", which is exactly what this column represents.\n\n"FunderAcronym" is the acronym of the funding organization, like "MRC" for Medical Research Council. The schema has "funder-id", which is an identifier from the Funder Registry. An acronym could serve as a concise identifier, so I\'ll map this to "funder-id".\n\n"OrganisationReference" seems to be a specific reference number for the organization, such as "BB/M018040/1". This aligns with "award-number" as well, but since "HRCS2022_I

In [1]:
import pandas as pd

In [3]:
!pip install beautifulsoup4

In [6]:
import requests
from bs4 import BeautifulSoup

# 2022 has ID 400
url = "https://www.hfsp.org/awardees/awards"
params = {
    "field_award_year_target_id[]": 400,
    "page": 1  # Page 1 = index 0
}

response = requests.get(url, params=params)
soup = BeautifulSoup(response.text, "html.parser")

# Find all award blocks
awards = soup.select(".views-row")

for award in awards:
    title = award.select_one(".field--name-title").get_text(strip=True)
    print(f"Award Title: {title}")

    # You can extract more fields like name, institution, country, etc.